# Problem Set 4 - Structural Estimation

## John Wilson
### July 15, 2017

In [4]:
import numpy as np
import scipy.stats as sts
from scipy.special import gamma, beta
from matplotlib import pyplot as plt
import scipy.optimize as opt
import scipy.integrate as integrate
%matplotlib notebook

## Problem 4

### Part a)

In [5]:
data = np.genfromtxt('data/MacroSeries.txt', delimiter=',')
c = data[:,0]
k = data[:,1]
w = data[:,2]
r = data[:,3]

In [7]:
def mom_1(z, rho, mu):
    moment = 0
    for i in range(len(z)-1):
        moment += z[i+1] - rho * z[i] - (1 - rho) * mu
    return moment / (len(z) - 1)

def mom_2(z, rho, mu):
    moment = 0
    for i in range(len(z)-1):
        moment += ((z[i+1] - rho * z[i] - (1 - rho) * mu) * z[i])
    return moment / (len(z) - 1)

def mom_3(beta, alpha, z, k, c):
    moment = 0
    for i in range(len(z)-1):
        moment += beta * alpha * np.exp(z[i+1]) * \
            k[i+1] ** (alpha - 1) * (c[i] / c[i+1]) - 1
    return moment / (len(z) - 1)

def mom_4(beta, alpha, z, k, c, w):
    moment = 0
    for i in range(len(z)-1):
        moment += (beta * alpha * np.exp(z[i+1]) * \
            k[i+1] ** (alpha - 1) * (c[i] / c[i+1]) - 1) * w[i]
    return moment / (len(z) - 1)

def calc_moments(alpha, beta, rho, mu, c, k, w, z):
    moment_1 = mom_1(z, rho, mu)
    moment_2 = mom_2(z, rho, mu)
    moment_3 = mom_3(beta, alpha, z, k, c)
    moment_4 = mom_4(beta, alpha, z, k, c, w)
    return np.array([moment_1, moment_2, moment_3, moment_4])

def criterion(params, *args):
    alpha, beta, rho, mu = params
    c, k, w, r = args
    z = np.log(r) - np.log(alpha) - (alpha- 1) * np.log(k)
    z[0] = mu
    err_vec = calc_moments(alpha, beta, rho, mu, c, k, w, z)
    return np.dot(err_vec, err_vec)

In [58]:
alpha_init = .2
beta_init = .6
rho_init = 0
mu_init = 10
init_vals = np.array([alpha_init, beta_init, rho_init, mu_init])
params = (c, k, w, r)
bounds = ((1e-10, 1 - 1e-10), (1e-10, 1 - 1e-10), \
          (-1 + 1e-10, 1 - 1e-10), (1e-10, None))
results = opt.minimize(criterion, init_vals, args = params, \
                      method = 'L-BFGS-B', bounds = bounds)

In [60]:
alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results.x
crit_val = results.fun
print("alpha_gmm =", alpha_GMM, " beta_gmm=", beta_GMM)
print("rho_gmm =", rho_GMM, " mu_gmm=", mu_GMM)
print("Criterion value: ", crit_val)

alpha_gmm = 0.41560732652  beta_gmm= 0.989999995966
rho_gmm = 0.9999999999  mu_gmm= 10.1241268785
Criterion value:  0.00194168036757
